In [ ]:
import pandas as pd

In [ ]:
d = {"苏暮雨": "雨",
     "苏昌河": "河",
     "白鹤淮": "淮",
     "苏喆": "喆",
     "慕雨墨": "墨",
     "唐怜月": "月",
     "慕雪薇": "薇",
     "慕青羊": "羊"}

df_role = pd.read_excel("暗河传.xlsx", sheet_name="页数", usecols="A:B, J:Q, S", na_filter = False).dropna()
df_date = pd.read_excel("暗河传.xlsx", sheet_name="页数", usecols="A:B, T:AD")

##### Check roles

In [ ]:
for index, row in df_role.iterrows():
    if row["角色"]:
        for name, aname in d.items():
            if name in row["角色"] and name + "（小）" not in row["角色"]:
                if row[aname] != 1:
                    print(row[0], row[1], aname, "should be 1")
            elif row[aname] == 1:
                print(row[0], row[1], aname, "should be 0")

##### Fill in roles

In [ ]:
for index, row in df_role.iterrows():
    if row["角色"]:
        for name, aname in d.items():
            if name in row["角色"] and name + "（小）" not in row["角色"]:
                df_role.loc[index, aname] = 1
            else:
                df_role.loc[index, aname] = 0

df_role.to_excel('ahz.xlsx', engine='xlsxwriter', sheet_name='按集数', index=False)

print("===== fill in complete =====")

##### Count dates

In [ ]:
dates = ["拍摄日期", "Unnamed: 20", "Unnamed: 21", "Unnamed: 22", "Unnamed: 23", "Unnamed: 24", "Unnamed: 25", "Unnamed: 26", "Unnamed: 27", "Unnamed: 28", "Unnamed: 29"]

for index, row in df_date.iterrows():
    if not pd.isna(row["拍摄日期"]):
        df_date.loc[index, "天数"] = 0
        for date in dates:
            if not pd.isna(df_date.loc[index, date]):
                df_date.loc[index, "天数"] += 1

df_count = df_date[["场次", "主要内容", "天数"]]
# df_count = df_count.fillna(0)
# df_count["天数"] = df_count["天数"].astype(int)
df_count = df_count.sort_values(by=["天数"], ascending=False)

df_count.to_excel("ahz_count_date.xlsx", index=False)
df_count[:30]